In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from pandas import ExcelWriter
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dense, Dropout, TimeDistributed, RNN, GRU
from keras.layers import Conv1D, MaxPooling1D, Flatten, Bidirectional
from keras.layers import Input, Flatten, Activation, Reshape, RepeatVector, Concatenate, Average
from keras.layers import Flatten, Convolution1D
from keras.layers import Dropout
from keras.layers import Activation, GlobalAveragePooling1D
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.models import Model
from sklearn import preprocessing
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, MaxPooling2D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import math
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import os

In [ ]:
def split_sequence(sequence, look_back, forecast_horizon):
 X, y = list(), list()
 for i in range(len(sequence)): 
   lag_end = i + look_back
   forecast_end = lag_end + forecast_horizon
   if forecast_end > len(sequence):
     break
   seq_x, seq_y = sequence[i:lag_end], sequence[lag_end:forecast_end]
   X.append(seq_x)
   y.append(seq_y)
 return np.array(X), np.array(y)

In [ ]:
def model_cnn_with_multy_head_3(frame, lags, razb):
    for s in range(0,len(razb)-1):
        vars()['scaler'+ '_'+str(s)]=preprocessing.MinMaxScaler() 
        vars()['frames'+ '_'+str(s)]=vars()['scaler'+ '_'+str(s)].fit_transform(frame.iloc[:,razb[s]:razb[s+1]])
        vars()['x_train_all'+ '_'+str(s)]=split_sequence(vars()['frames'+ '_'+str(s)],lags,1)[0]
        vars()['y_train_all'+ '_'+str(s)]=split_sequence(vars()['frames'+ '_'+str(s)],lags,1)[1]

    in_layers, out_layers = list(), list()   
    num_outputs=0
    for s in range(0,len(razb)-1):
        inputs = Input(shape=(vars()['x_train_all'+ '_'+str(s)].shape[1],vars()['x_train_all'+ '_'+str(s)].shape[2]))
        conv2 = Conv1D(filters=vars()['x_train_all'+ '_'+str(s)].shape[2],
                           kernel_size=vars()['x_train_all'+ '_'+str(s)].shape[1], activation='tanh')(inputs)
        conv2 = Conv1D(filters=vars()['x_train_all'+ '_'+str(s)].shape[1], kernel_size=1, activation='tanh')(conv2)
        flat = Flatten()(conv2)
        in_layers.append(inputs)
        out_layers.append(flat)
        num_outputs=num_outputs+vars()['x_train_all'+ '_'+str(s)].shape[2]

        # merge heads
    merged = Concatenate(axis=1)(out_layers)
        # interpretation

    x2 = Reshape((1, merged.shape[1]))(merged)
    x2 = Conv1D(merged.shape[1], 1, activation = 'tanh')(x2)
    x2 = Flatten()(x2)

    outputs = Dense(num_outputs)(x2)
    model = Model(inputs=in_layers, outputs=outputs)
    model.compile(loss='mae', optimizer='adam')
    return model

def model_cnn_with_multy_head_4(frame, lags, razb):
    for s in range(0,len(razb)-1):
        vars()['scaler'+ '_'+str(s)]=preprocessing.MinMaxScaler() 
        vars()['frames'+ '_'+str(s)]=vars()['scaler'+ '_'+str(s)].fit_transform(frame.iloc[:,razb[s]:razb[s+1]])
        vars()['x_train_all'+ '_'+str(s)]=split_sequence(vars()['frames'+ '_'+str(s)],lags,1)[0]
        vars()['y_train_all'+ '_'+str(s)]=split_sequence(vars()['frames'+ '_'+str(s)],lags,1)[1]

    in_layers, out_layers = list(), list()   
    num_outputs=0
    for s in range(0,len(razb)-1):
        inputs = Input(shape=(vars()['x_train_all'+ '_'+str(s)].shape[1],vars()['x_train_all'+ '_'+str(s)].shape[2]))
        conv2 = Conv1D(filters=vars()['x_train_all'+ '_'+str(s)].shape[2],
                           kernel_size=vars()['x_train_all'+ '_'+str(s)].shape[1], activation='tanh')(inputs)
#         conv2 = Conv1D(filters=vars()['x_train_all'+ '_'+str(s)].shape[1], kernel_size=1, activation='tanh')(conv2)
        flat = Flatten()(conv2)
        in_layers.append(inputs)
        out_layers.append(flat)
        num_outputs=num_outputs+vars()['x_train_all'+ '_'+str(s)].shape[2]

        # merge heads
    merged = Average()(out_layers)
    outputs = Dense(num_outputs)(merged)
    model = Model(inputs=in_layers, outputs=outputs)
    model.compile(loss='mae', optimizer='adam')
    return model

def model_cnn_with_multy_head_5(frame, lags, razb):
    for s in range(0,len(razb)-1):
        vars()['scaler'+ '_'+str(s)]=preprocessing.MinMaxScaler() 
        vars()['frames'+ '_'+str(s)]=vars()['scaler'+ '_'+str(s)].fit_transform(frame.iloc[:,razb[s]:razb[s+1]])
        vars()['x_train_all'+ '_'+str(s)]=split_sequence(vars()['frames'+ '_'+str(s)],lags,1)[0]
        vars()['y_train_all'+ '_'+str(s)]=split_sequence(vars()['frames'+ '_'+str(s)],lags,1)[1]

    in_layers, out_layers = list(), list()   
    num_outputs=0
    for s in range(0,len(razb)-1):
        inputs = Input(shape=(vars()['x_train_all'+ '_'+str(s)].shape[1],vars()['x_train_all'+ '_'+str(s)].shape[2]))
        conv2 = Conv1D(filters=vars()['x_train_all'+ '_'+str(s)].shape[2],
                           kernel_size=vars()['x_train_all'+ '_'+str(s)].shape[1], activation='tanh')(inputs)
#         conv2 = Conv1D(filters=vars()['x_train_all'+ '_'+str(s)].shape[1], kernel_size=1, activation='tanh')(conv2)
        flat = Flatten()(conv2)
        in_layers.append(inputs)
        out_layers.append(flat)
        num_outputs=num_outputs+vars()['x_train_all'+ '_'+str(s)].shape[2]

    merged = Concatenate(axis=1)(out_layers)
    x2 = Reshape((1, merged.shape[1]))(merged)
    x2 = Conv1D(merged.shape[1], 1, activation = 'tanh')(x2)
    x2 = Flatten()(x2)

    outputs = Dense(num_outputs)(x2)
    model = Model(inputs=in_layers, outputs=outputs)
    model.compile(loss='mae', optimizer='adam')
    return model

def model_cnn_with_multy_head_6(frame, lags, razb):
    for s in range(0,len(razb)-1):
        vars()['scaler'+ '_'+str(s)]=preprocessing.MinMaxScaler() 
        vars()['frames'+ '_'+str(s)]=vars()['scaler'+ '_'+str(s)].fit_transform(frame.iloc[:,razb[s]:razb[s+1]])
        vars()['x_train_all'+ '_'+str(s)]=split_sequence(vars()['frames'+ '_'+str(s)],lags,1)[0]
        vars()['y_train_all'+ '_'+str(s)]=split_sequence(vars()['frames'+ '_'+str(s)],lags,1)[1]

    in_layers, out_layers = list(), list()   
    num_outputs=0
    for s in range(0,len(razb)-1):
        inputs = Input(shape=(vars()['x_train_all'+ '_'+str(s)].shape[1],vars()['x_train_all'+ '_'+str(s)].shape[2]))
        conv2 = Conv1D(filters=vars()['x_train_all'+ '_'+str(s)].shape[2],
                           kernel_size=vars()['x_train_all'+ '_'+str(s)].shape[1], activation='tanh')(inputs)
#         conv2 = Conv1D(filters=vars()['x_train_all'+ '_'+str(s)].shape[1], kernel_size=1, activation='tanh')(conv2)
        flat = Flatten()(conv2)
        in_layers.append(inputs)
        out_layers.append(flat)
        num_outputs=num_outputs+vars()['x_train_all'+ '_'+str(s)].shape[2]

    merged = Concatenate(axis=1)(out_layers)
    x2 = Reshape((1, merged.shape[1]))(merged)
    x2 = Conv1D(lags, 1, activation = 'tanh')(x2)
    x2 = Flatten()(x2)

    outputs = Dense(num_outputs)(x2)
    model = Model(inputs=in_layers, outputs=outputs)
    model.compile(loss='mae', optimizer='adam')
    return model

def model_cnn_with_multy_head_7(frame, lags, razb):
    for s in range(0,len(razb)-1):
        vars()['scaler'+ '_'+str(s)]=preprocessing.MinMaxScaler() 
        vars()['frames'+ '_'+str(s)]=vars()['scaler'+ '_'+str(s)].fit_transform(frame.iloc[:,razb[s]:razb[s+1]])
        vars()['x_train_all'+ '_'+str(s)]=split_sequence(vars()['frames'+ '_'+str(s)],lags,1)[0]
        vars()['y_train_all'+ '_'+str(s)]=split_sequence(vars()['frames'+ '_'+str(s)],lags,1)[1]

    in_layers, out_layers = list(), list()   
    num_outputs=0
    for s in range(0,len(razb)-1):
        inputs = Input(shape=(vars()['x_train_all'+ '_'+str(s)].shape[1],vars()['x_train_all'+ '_'+str(s)].shape[2]))
        conv2 = Conv1D(filters=vars()['x_train_all'+ '_'+str(s)].shape[2],
                           kernel_size=vars()['x_train_all'+ '_'+str(s)].shape[1], activation='tanh')(inputs)
#         conv2 = Conv1D(filters=vars()['x_train_all'+ '_'+str(s)].shape[1], kernel_size=1, activation='tanh')(conv2)
        flat = Flatten()(conv2)
        in_layers.append(inputs)
        out_layers.append(flat)
        num_outputs=num_outputs+vars()['x_train_all'+ '_'+str(s)].shape[2]

    merged = Concatenate(axis=1)(out_layers)
    x2 = Reshape((1, merged.shape[1]))(merged)
    x2 = LSTM(64, activation = 'tanh')(x2)
    x2 = Flatten()(x2)

    outputs = Dense(num_outputs)(x2)
    model = Model(inputs=in_layers, outputs=outputs)
    model.compile(loss='mae', optimizer='adam')
    return model

# Average()(xs_lst)

In [ ]:
tf.keras.utils.plot_model(model_cnn_with_multy_head_7(frame, lags, razb), show_shapes=True)

In [ ]:
def multy_head_all(frame, lags, razb, batch_size, num_architecture):
    for s in range(0,len(razb)-1):
        vars()['scaler'+ '_'+str(s)]=preprocessing.MinMaxScaler() 
        vars()['frames'+ '_'+str(s)]=vars()['scaler'+ '_'+str(s)].fit_transform(frame.iloc[:,razb[s]:razb[s+1]])
        vars()['x_train_all'+ '_'+str(s)]=split_sequence(vars()['frames'+ '_'+str(s)],lags,1)[0]
        vars()['y_train_all'+ '_'+str(s)]=split_sequence(vars()['frames'+ '_'+str(s)],lags,1)[1]

    itog_fr_x=vars()['x_train_all'+ '_'+str(0)]
    itog_fr_y=vars()['y_train_all'+ '_'+str(0)]
    for s in range(1,len(razb)-1):
        itog_fr_x=[itog_fr_x,vars()['x_train_all'+ '_'+str(s)]]
        itog_fr_y=[itog_fr_y,vars()['y_train_all'+ '_'+str(s)]]

    if num_architecture==0:
        model = model_cnn_with_multy_head_0(frame, lags, razb)
    if num_architecture==1:
        model = model_cnn_with_multy_head_1(frame, lags, razb)
    if num_architecture==2:
        model = model_cnn_with_multy_head_2(frame, lags, razb)
    if num_architecture==3:
        model = model_cnn_with_multy_head_3(frame, lags, razb)
    if num_architecture==4:
        model = model_cnn_with_multy_head_4(frame, lags, razb)
    if num_architecture==5:
        model = model_cnn_with_multy_head_5(frame, lags, razb)
    if num_architecture==6:
        model = model_cnn_with_multy_head_6(frame, lags, razb)
    if num_architecture==7:
        model = model_cnn_with_multy_head_7(frame, lags, razb)

    early_stop = EarlyStopping(monitor = 'loss', patience = 10)



    model.fit(itog_fr_x,itog_fr_y, 
                                    epochs=50, 
                                    batch_size=batch_size, #от размера батчей зависит точность модели, очень странная хуйня
                                    verbose=0,
                                    callbacks=[early_stop],
                                    validation_split=0
    #                                 shuffle=False,
                                    )   
    return model

def sm_wind_multy_head(frame,lags,razb,batch_size, num_architecture):
    prog_frame=pd.DataFrame(columns=frame.columns)
    orig_frame=pd.DataFrame(columns=frame.columns)
    iteratation_5=int((len(frame)-399)/5)+1
#     iteratation_5=50
    for i in range(0,iteratation_5):
        model=multy_head_all(frame.iloc[0+5*i:399+5*i,:], lags,razb,batch_size, num_architecture)
        frames=frame.iloc[0+5*i:399+1+5*i,:]

        s=0
        vars()['scaler'+ '_'+str(s)]=preprocessing.MinMaxScaler() 
        vars()['frames'+ '_'+str(s)]=vars()['scaler'+ '_'+str(s)].fit_transform(frames.iloc[:,razb[s]:razb[s+1]])
        vars()['x_test_all'+ '_'+str(s)]=split_sequence(vars()['frames'+ '_'+str(s)],lags,1)[0]
        vars()['y_test_all'+ '_'+str(s)]=split_sequence(vars()['frames'+ '_'+str(s)],lags,1)[1]
        vars()['x_input'+ '_'+str(s)]=vars()['x_test_all'+ '_'+str(s)][-1]
        vars()['x_input'+ '_'+str(s)]=vars()['x_input'+ '_'+str(s)].reshape((1,
                                                                                 vars()['x_test_all'+ '_'+str(s)].shape[1],
                                                                                 vars()['x_test_all'+ '_'+str(s)].shape[2]))
        x_input=vars()['x_input'+ '_'+str(s)]


        for s in range(1,len(frame.columns)):
        #     print(s)
            vars()['scaler'+ '_'+str(s)]=preprocessing.MinMaxScaler() 
            vars()['frames'+ '_'+str(s)]=vars()['scaler'+ '_'+str(s)].fit_transform(frames.iloc[:,razb[s]:razb[s+1]])
            vars()['x_test_all'+ '_'+str(s)]=split_sequence(vars()['frames'+ '_'+str(s)],lags,1)[0]
            vars()['y_test_all'+ '_'+str(s)]=split_sequence(vars()['frames'+ '_'+str(s)],lags,1)[1]

            vars()['x_input'+ '_'+str(s)]=vars()['x_test_all'+ '_'+str(s)][-1]
            vars()['x_input'+ '_'+str(s)]=vars()['x_input'+ '_'+str(s)].reshape((1,
                                                                                 vars()['x_test_all'+ '_'+str(s)].shape[1],
                                                                                 vars()['x_test_all'+ '_'+str(s)].shape[2]))

            x_input=[x_input,vars()['x_input'+ '_'+str(s)]]


        # i=0
        yhat = model.predict(x_input, verbose=0)
        s=0
        prog_all=[]
        orig_all=[]
        vars()['testPredict'+ '_'+str(s)] = vars()['scaler'+ '_'+str(s)].inverse_transform(yhat[0:1,s:(s+1)])[0] 
        vars()['testY'+ '_'+str(s)]=vars()['scaler'+ '_'+str(s)].inverse_transform(vars()['y_test_all'+ '_'+str(s)][-1])
        prog_all.append(vars()['testPredict'+ '_'+str(s)])
        orig_all.append(vars()['testY'+ '_'+str(s)][0])
        for s in range(1,len(frame.columns)):
            vars()['testPredict'+ '_'+str(s)] = vars()['scaler'+ '_'+str(s)].inverse_transform(yhat[0:1,s:(s+1)])[0] 
            vars()['testY'+ '_'+str(s)]=vars()['scaler'+ '_'+str(s)].inverse_transform(vars()['y_test_all'+ '_'+str(s)][-1])
            prog_all.append(vars()['testPredict'+ '_'+str(s)])
            orig_all.append(vars()['testY'+ '_'+str(s)][0])
        prog_frame.loc[i]=prog_all
        orig_frame.loc[i]=orig_all
    return prog_frame, orig_frame

def povtor_multy_head(frame,lags,razb,batch_size, num_architecture, povtor):
    for i in range(0,len(frame.columns)):
        vars()['frrr'+ '_'+str(i)]=pd.DataFrame()

    for p in range(0,povtor):
        cc=sm_wind_multy_head(frame,lags,razb,batch_size, num_architecture)
        for s in range(len(cc[0].columns)):

            zz=[]
            for i in range(0,len(cc[0].iloc[:,s])):
                zz.append(cc[0].iloc[i,s][0])
            vars()['frrr'+ '_'+str(s)][str(p)]=np.exp(zz)

    pogr=[]

    s=0
    vars()['orig'+ '_'+str(s)]=[]        
    for i in range(0,len(cc[1].iloc[:,s])):
        vars()['orig'+ '_'+str(s)].append(cc[1].iloc[i,s][0])
    vars()['orig'+ '_'+str(s)]=np.exp(vars()['orig'+ '_'+str(s)])
    vars()['prog_mean'+ '_'+str(s)]=vars()['frrr'+ '_'+str(s)].transpose().mean()
    pogr.append((100*abs(vars()['orig'+ '_'+str(s)]-vars()['prog_mean'+ '_'+str(s)])/vars()['orig'+ '_'+str(s)]).mean())
    all_orig=vars()['orig'+ '_'+str(s)]
    all_prog_mean=vars()['prog_mean'+ '_'+str(s)]

    for s in range(1,len(cc[0].columns)): 

        vars()['orig'+ '_'+str(s)]=[]        
        for i in range(0,len(cc[1].iloc[:,s])):
            vars()['orig'+ '_'+str(s)].append(cc[1].iloc[i,s][0])
        vars()['orig'+ '_'+str(s)]=np.exp(vars()['orig'+ '_'+str(s)])
        vars()['prog_mean'+ '_'+str(s)]=vars()['frrr'+ '_'+str(s)].transpose().mean()
        pogr.append((100*abs(vars()['orig'+ '_'+str(s)]-vars()['prog_mean'+ '_'+str(s)])/vars()['orig'+ '_'+str(s)]).mean())
        all_orig=[all_orig,vars()['orig'+ '_'+str(s)]]
        all_prog_mean=[all_prog_mean,vars()['prog_mean'+ '_'+str(s)]]
    
    return pogr, all_orig, all_prog_mean

In [ ]:

os.chdir("C:/Users/VIP13/Статья 2/Котировки 6")
frame_3=pd.read_csv('all_snp_btc_rv_rr.csv', delimiter=';')
frame=frame_3.iloc[:,1:].copy() #на входе подаются квадраты волатильностей, поэтому сначала из них берем корни, а потом логарифмируем
frame=frame.loc[:, [c for c in frame.columns if (frame[c]>0).all()]]
frame=np.log(np.sqrt(frame))
ind=[i for i in range(0,len(frame))]
frame.index=ind
razb=[0,1,3]
lags=5

cc0=povtor_multy_head(frame,5,razb,5, 4, 10)
print(cc0[0])
cc1=povtor_multy_head(frame,5,razb,5, 5, 10)
print(cc1[0])
cc2=povtor_multy_head(frame,5,razb,5, 6, 10)
print(cc2[0])
cc3=povtor_multy_head(frame,5,razb,5, 7, 10)
print(cc3[0])